In [26]:
# williams_macd_rank_or_fcfs_rsi_vol.py
# Daily strategy with optional ranking, RSI filter, and volume breakout.
# Entry (hard): MACD cross-up + W%R cross-up from < -80 + MACD hist > 0 and rising.
# Optional filters: RSI >= threshold, Volume breakout vs MA.
# Exit: daily technical gate (MACD/W%R) + strict stop-loss. No min-hold/grace.
# If RANKING_ENABLED=False -> "first come first serve" up to cap, no rank exits.

import os, re, math, datetime, time
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional
from collections.abc import Iterable
import numpy as np
import pandas as pd
import yfinance as yf

# ---------------- Logging ----------------
def log(msg: str):
    ts = datetime.datetime.now().strftime("%H:%M:%S")
    print(f"[{ts}] {msg}", flush=True)

def fmt_metrics(m: Dict) -> Dict:
    out = {}
    for k, v in (m or {}).items():
        if isinstance(v, (np.floating, np.integer)):
            v = v.item()
        out[k] = v
    return out

# ============== CONFIG ===================
OUTDIR   = "outputs"
DATADIR  = "data"
FORCE_REFRESH = False

# ---- Universe (replace with your full list) ----
UNIVERSE = ['360ONE.NS', '3MINDIA.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABLBL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ACMESOLAR.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'AEGISLOG.NS', 'AEGISVOPAK.NS', 'AFCONS.NS', 'AFFLE.NS', 'AGARWALEYE.NS', 'AIAENG.NS', 'AIIL.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'AKZOINDIA.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ARE&M.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATHERENERG.NS', 'ATUL.NS', 'AUBANK.NS', 'AUROPHARMA.NS', 'AWL.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJAJHFL.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'BASF.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBTC.NS', 'BDL.NS', 'BEL.NS', 'BEML.NS', 'BERGEPAINT.NS', 'BHARATFORG.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BHEL.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUEJET.NS', 'BLUESTARCO.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'BSE.NS', 'BSOFT.NS', 'CAMPUS.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CAPLIPOINT.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOICEIN.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIPLA.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CONCOR.NS', 'CONCORDBIO.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CRISIL.NS', 'CROMPTON.NS', 'CUB.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DBREALTY.NS', 'DCMSHRIRAM.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'DLF.NS', 'DMART.NS', 'DOMS.NS', 'DRREDDY.NS', 'ECLERX.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ENRIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'FACT.NS', 'FEDERALBNK.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FIRSTCRY.NS', 'FIVESTAR.NS', 'FLUOROCHEM.NS', 'FORCEMOT.NS', 'FORTIS.NS', 'FSL.NS', 'GAIL.NS', 'GESHIP.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GMDCLTD.NS', 'GMRAIRPORT.NS', 'GODFRYPHLP.NS', 'GODIGIT.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GPIL.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GRSE.NS', 'GSPL.NS', 'GUJGASLTD.NS', 'GVT&D.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEG.NS', 'HEROMOTOCO.NS', 'HEXT.NS', 'HFCL.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HSCL.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDEA.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'IFCI.NS', 'IGIL.NS', 'IGL.NS', 'IIFL.NS', 'IKS.NS', 'INDGN.NS', 'INDHOTEL.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIGO.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFY.NS', 'INOXINDIA.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'IOB.NS', 'IOC.NS', 'IPCALAB.NS', 'IRB.NS', 'IRCON.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'ITC.NS', 'ITCHOTELS.NS', 'ITI.NS', 'J&KBANK.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JKCEMENT.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JPPOWER.NS', 'JSL.NS', 'JSWENERGY.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KAJARIACER.NS', 'KALYANKJIL.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KEI.NS', 'KFINTECH.NS', 'KIMS.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KPIL.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'KSB.NS', 'LALPATHLAB.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICHSGFIN.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LT.NS', 'LTF.NS', 'LTFOODS.NS', 'LTIM.NS', 'LTTS.NS', 'LUPIN.NS', 'M&M.NS', 'M&MFIN.NS', 'MAHABANK.NS', 'MAHSCOOTER.NS', 'MAHSEAMLES.NS', 'MANAPPURAM.NS', 'MANKIND.NS', 'MANYAVAR.NS', 'MAPMYINDIA.NS', 'MARICO.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MCX.NS', 'MEDANTA.NS', 'METROPOLIS.NS', 'MFSL.NS', 'MGL.NS', 'MINDACORP.NS', 'MMTC.NS', 'MOTHERSON.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MRF.NS', 'MRPL.NS', 'MSUMI.NS', 'MUTHOOTFIN.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NBCC.NS', 'NCC.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIVABUPA.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPC.NS', 'NTPCGREEN.NS', 'NUVAMA.NS', 'NUVOCO.NS', 'NYKAA.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'ONESOURCE.NS', 'ONGC.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PAYTM.NS', 'PCBL.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PGHH.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PNB.NS', 'PNBHOUSING.NS', 'POLICYBZR.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POONAWALLA.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'PPLPHARMA.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PTCIL.NS', 'PVRINOX.NS', 'RADICO.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RAMCOCEM.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RELINFRA.NS', 'RHIM.NS', 'RITES.NS', 'RKFORGE.NS', 'RPOWER.NS', 'RRKABEL.NS', 'RVNL.NS', 'SAGILITY.NS', 'SAIL.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'SRF.NS', 'STARHEALTH.NS', 'SUMICHEM.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TARIL.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TBOTEK.NS', 'TCS.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'THELEELA.NS', 'THERMAX.NS', 'TIINDIA.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TTML.NS', 'TVSMOTOR.NS', 'UBL.NS', 'UCOBANK.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UNITDSPR.NS', 'UNOMINDA.NS', 'UPL.NS', 'USHAMART.NS', 'UTIAMC.NS', 'VBL.NS', 'VEDL.NS', 'VENTIVE.NS', 'VGUARD.NS', 'VIJAYA.NS', 'VMM.NS', 'VOLTAS.NS', 'VTL.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ZFCVINDIA.NS', 'ZYDUSLIFE.NS']
TRAIN_START = "2015-01-01"
TRAIN_END   = "2020-12-31"
TEST_START  = "2021-01-01"
TEST_END    = "2025-07-01"

STARTING_CAPITAL = 200_000.0
SLIPPAGE_BPS     = 00
APPLY_FEES       = True  # uses Groww NSE delivery model (no UPI mandate)

# ---- Portfolio capacity (used in both modes) ----
RANK_TOP_N             = 4   # acts as capacity cap even if ranking is disabled

# ---- Ranking mode toggle ----
RANKING_ENABLED        = False    # << Toggle here
REBALANCE_FREQ         = "W-FRI" # "W-FRI","2W-FRI","3W-FRI","M"
RANK_EXIT_THRESHOLD    = 40      # ignored if ranking disabled
RANK_LOOKBACK_DAYS     = 252
WITHIN_52W             = 0.20    # eligible if Close >= (1-WITHIN_52W)*52w high AND Close > SMA200

# ---- Option: protect bullish holdings from rank exit (only when ranking enabled) ----
PRIORITIZE_BULLISH_ON_REBAL = True
BULLISH_HOLD_REQUIRE_LINE_GT_SIG  = True
BULLISH_HOLD_REQUIRE_HIST_POS     = True
BULLISH_HOLD_REQUIRE_HIST_RISE    = False
BULLISH_HOLD_WILLR_LEVEL          = -50

# ---- Entry bundle (hard) ----
WILLR_N          = 14
WILLR_ENTER      = -80
WILLR_EXIT       = -20
MACD_FAST        = 12
MACD_SLOW        = 26
MACD_SIGNAL      = 9

# ---- Daily exit gate ----
USE_WR_MACD_EXITGATE = True
EXIT_REQUIRE_CROSS   = True    # cross-down semantics
EXIT_GATE_OR         = True    # OR between MACD & W%R exit

# ---- Intraday stop-loss ----
USE_STOP_LOSS = True
STOP_LOSS_PCT = 0.04           # strict 4% stop (configurable)

# ---- Optional RSI & Volume filters (toggles) ----
USE_RSI_FILTER   = False
RSI_LEN          = 14
RSI_MIN          = 55.0        # require RSI >= 55

USE_VOL_BREAKOUT = False
VOL_LOOKBACK     = 20
VOL_MULTIPLIER   = 1.5         # require Volume >= 1.5 × VOL_MA

# ---- Optional 4h confirm (kept available; OFF by default) ----
USE_MTF_CONFIRM  = False
MTF_STRICT       = False

os.makedirs(OUTDIR, exist_ok=True)
os.makedirs(DATADIR, exist_ok=True)

# ============== Helpers & Data Cache ==============
def sanitize_ticker(t) -> str:
    return re.sub(r"[^A-Za-z0-9._-]", "_", str(t))

def _flatten_tickers(tickers):
    if isinstance(tickers, (str, bytes)): return [str(tickers)]
    flat=[]
    for x in tickers:
        if isinstance(x, (str, bytes)): flat.append(str(x))
        elif isinstance(x, Iterable):
            for y in x: flat.append(str(y))
        else:
            flat.append(str(x))
    flat=[t for t in flat if t and t.strip()]
    seen=set(); out=[]
    for t in flat:
        if t not in seen: out.append(t); seen.add(t)
    return out

def yf_download(ticker: str, start: str, end: str, attempts: int=3, sleep_s: float=0.8) -> pd.DataFrame:
    end_inc = (pd.to_datetime(end) + pd.Timedelta(days=1)).strftime("%Y-%m-%d")
    for k in range(1, attempts+1):
        try:
            df = yf.download(ticker, start=start, end=end_inc, auto_adjust=True,
                             progress=False, multi_level_index=False)
            if df is not None and not df.empty:
                df = df.rename(columns=lambda c: c.strip().title())
                if "Adj Close" not in df.columns and "Close" in df.columns:
                    df["Adj Close"] = df["Close"]
                if "Volume" not in df.columns:
                    df["Volume"] = 0
                keep = [c for c in ["Open","High","Low","Close","Adj Close","Volume"] if c in df.columns]
                df = df[keep]
                df.index = pd.to_datetime(df.index).normalize()
                df = df[~df.index.duplicated(keep="last")]
                return df.sort_index()
        except Exception:
            pass
        if k < attempts: time.sleep(sleep_s)
    return pd.DataFrame()

def load_or_download_prices(tickers, start: str, end: str, force_refresh: bool=False) -> Dict[str,pd.DataFrame]:
    tickers = _flatten_tickers(tickers)
    data = {}; missing = []; extended_head = 0; extended_tail = 0
    for i, t in enumerate(tickers, start=1):
        fn = os.path.join(DATADIR, f"{sanitize_ticker(t)}.csv")
        if force_refresh or (not os.path.exists(fn)):
            if i % 25 == 0: log(f"[cache] downloading {i}/{len(tickers)}: {t}")
            df = yf_download(t, start, end)
            if df.empty: missing.append(t); continue
            df.to_csv(fn, index=True); data[t] = df; continue
        df_local = pd.read_csv(fn, index_col=0, parse_dates=True).sort_index()
        df_local.index = pd.to_datetime(df_local.index).normalize()
        if "Adj Close" not in df_local.columns and "Close" in df_local.columns:
            df_local["Adj Close"] = df_local["Close"]
        if "Volume" not in df_local.columns:
            df_local["Volume"] = 0
        have_min, have_max = df_local.index.min(), df_local.index.max()
        want_min, want_max = pd.to_datetime(start), pd.to_datetime(end)
        if have_min > want_min:
            df_head = yf_download(t, start, (have_min - pd.Timedelta(days=1)).strftime("%Y-%m-%d"))
            if not df_head.empty:
                df_local = pd.concat([df_head, df_local]).sort_index(); extended_head += 1
        if have_max < want_max:
            df_tail = yf_download(t, (have_max + pd.Timedelta(days=1)).strftime("%Y-%m-%d"), end)
            if not df_tail.empty:
                df_local = pd.concat([df_local, df_tail]).sort_index(); extended_tail += 1
        if df_local.empty: missing.append(t); continue
        df_local.to_csv(fn, index=True)
        data[t] = df_local.loc[(df_local.index >= want_min) & (df_local.index <= want_max)]
    log(f"[cache] requested: {len(tickers)}, loaded: {len(data)}, missing: {len(missing)}, extended_head: {extended_head}, extended_tail: {extended_tail}")
    if missing:
        pd.DataFrame({"ticker": missing}).to_csv(os.path.join(OUTDIR,"missing_tickers.csv"), index=False)
        log(f"[cache] wrote list of missing/empty tickers.")
    pd.DataFrame({"ticker": list(data.keys())}).to_csv(os.path.join(OUTDIR,"loaded_tickers.csv"), index=False)
    return data

# ============== Indicators =================
def sma(s: pd.Series, n: int) -> pd.Series:
    return s.rolling(n, min_periods=n).mean()

def ema(s: pd.Series, span: int) -> pd.Series:
    return s.ewm(span=span, adjust=False, min_periods=span).mean()

def macd_parts(close: pd.Series, fast: int, slow: int, signal: int):
    e_f = ema(close, fast); e_s = ema(close, slow)
    macd_line = e_f - e_s
    macd_signal = ema(macd_line, signal)
    macd_hist = macd_line - macd_signal
    return macd_line, macd_signal, macd_hist

def williams_r(h: pd.Series, l: pd.Series, c: pd.Series, n: int) -> pd.Series:
    hh = h.rolling(n, min_periods=n).max()
    ll = l.rolling(n, min_periods=n).min()
    return -100 * (hh - c) / (hh - ll)

def rsi_wilder(close: pd.Series, n: int=14) -> pd.Series:
    delta = close.diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    roll_up = up.ewm(alpha=1/n, adjust=False).mean()
    roll_down = down.ewm(alpha=1/n, adjust=False).mean()
    rs = roll_up / roll_down.replace(0, np.nan)
    rsi = 100 - (100 / (1 + rs))
    return rsi

def compute_indicators_df(price_df: pd.DataFrame, rank_lookback: int) -> pd.DataFrame:
    close, high, low, vol = price_df["Close"], price_df["High"], price_df["Low"], price_df["Volume"]
    macd_line, macd_sig, macd_hist = macd_parts(close, MACD_FAST, MACD_SLOW, MACD_SIGNAL)
    out = {
        "SMA_200":  sma(close, 200),
        "HIGH_252": high.rolling(252, min_periods=252).max(),
        "RETN":     (close / close.shift(rank_lookback)) - 1.0,
        "MACD_LINE": macd_line,
        "MACD_SIG":  macd_sig,
        "MACD_HIST": macd_hist,
        "WILLR":     williams_r(high, low, close, WILLR_N),
        "RSI":       rsi_wilder(close, RSI_LEN),
        "VOL_MA":    vol.rolling(VOL_LOOKBACK, min_periods=VOL_LOOKBACK).mean(),
    }
    return pd.DataFrame(out, index=price_df.index)

def load_or_build_indicators_for_ticker(ticker: str, price_df: pd.DataFrame, rank_lookback: int, force_refresh: bool=False) -> pd.DataFrame:
    ind_fn = os.path.join(DATADIR, f"indicators_{sanitize_ticker(ticker)}.csv")
    need_cols = ["SMA_200","HIGH_252","RETN","MACD_LINE","MACD_SIG","MACD_HIST","WILLR","RSI","VOL_MA"]
    need_rebuild = force_refresh or (not os.path.exists(ind_fn))
    if not need_rebuild:
        try:
            ind = pd.read_csv(ind_fn, index_col=0, parse_dates=True)
            ind.index = pd.to_datetime(ind.index).normalize()
            if (ind.index.max() >= price_df.index.max()) and all(c in ind.columns for c in need_cols):
                return ind.reindex(price_df.index)
            else:
                need_rebuild = True
        except Exception:
            need_rebuild = True
    log(f"[ind-cache] computing indicators for {ticker}")
    ind = compute_indicators_df(price_df, rank_lookback)
    ind.to_csv(ind_fn, index=True)
    return ind.reindex(price_df.index)

def build_indicator_caches(price_map: Dict[str,pd.DataFrame], rank_lookback: int, force_refresh: bool=False) -> Dict[str,Dict[str,pd.Series]]:
    log("Building/loading indicator caches for all symbols...")
    caches = {}
    for t, df in price_map.items():
        ind_df = load_or_build_indicators_for_ticker(t, df, rank_lookback, force_refresh=force_refresh)
        caches[t] = {col: ind_df[col] for col in ind_df.columns}
    log(f"Indicator caches ready for {len(caches)} symbols.")
    return caches

# ============== Groww Fees (NSE Delivery) ==============
def calc_fees(turnover_buy: float, turnover_sell: float) -> float:
    if not APPLY_FEES: return 0.0
    # Brokerage 0.1% each leg, min ₹5, max ₹20 per leg
    BROKER_PCT = 0.001
    BROKER_MIN = 5.0
    BROKER_CAP = 20.0
    # Taxes & charges
    STT_PCT = 0.001            # on buy & sell
    STAMP_BUY_PCT = 0.00015    # on buy only
    EXCH_PCT = 0.0000297
    SEBI_PCT = 0.000001
    IPFT_PCT = 0.000001
    GST_PCT = 0.18
    DP_SELL = 20.0 if turnover_sell >= 100 else 0.0

    def _broker(turnover):
        if turnover <= 0: return 0.0
        fee = turnover * BROKER_PCT
        fee = max(BROKER_MIN, min(fee, BROKER_CAP))
        return fee

    br_buy  = _broker(turnover_buy)
    br_sell = _broker(turnover_sell)
    stt   = STT_PCT * (turnover_buy + turnover_sell)
    stamp = STAMP_BUY_PCT * turnover_buy
    exch  = EXCH_PCT * (turnover_buy + turnover_sell)
    sebi  = SEBI_PCT * (turnover_buy + turnover_sell)
    ipft  = IPFT_PCT * (turnover_buy + turnover_sell)
    dp    = DP_SELL
    gst_base = br_buy + br_sell + dp + exch + sebi + ipft
    gst   = GST_PCT * gst_base
    return float((br_buy + br_sell) + stt + stamp + exch + sebi + ipft + dp + gst)

# ============== Rebalance scheduling (for ranking mode) ==============
def compute_rebalance_dates(dates: pd.DatetimeIndex, freq: str) -> List[pd.Timestamp]:
    dates = pd.DatetimeIndex(dates).sort_values()
    if freq == "M":
        temp = pd.Series(1, index=dates)
        return list(pd.to_datetime(temp.groupby(dates.to_period("M")).apply(lambda s: s.index.max()).values))
    m = re.match(r"^(\d*)W-([A-Z]{3})$", freq.upper())
    if not m: raise ValueError(f"Unsupported REBALANCE_FREQ: {freq}")
    n = int(m.group(1)) if m.group(1) else 1
    anchor = m.group(2)
    temp = pd.Series(1, index=dates)
    weekly_ends = list(pd.to_datetime(temp.groupby(dates.to_period(f"W-{anchor}")).apply(lambda s: s.index.max()).values))
    return weekly_ends if n <= 1 else weekly_ends[::n]

# ============== Ranking (for ranking mode) ==============
def build_rank_table(date_d: pd.Timestamp,
                     data: Dict[str,pd.DataFrame],
                     caches: Dict[str,Dict[str,pd.Series]]) -> pd.DataFrame:
    rows=[]
    for t, df in data.items():
        if date_d not in df.index: continue
        c = float(df.at[date_d, "Close"])
        try:
            high252 = float(caches[t]["HIGH_252"].loc[date_d])
            sma200  = float(caches[t]["SMA_200"].loc[date_d])
            retN    = float(caches[t]["RETN"].loc[date_d])
        except Exception:
            continue
        if any(map(np.isnan, [c, high252, sma200, retN])): continue
        within = (c >= (1.0 - WITHIN_52W) * high252)
        above  = (c > sma200)
        eligible = bool(within and above)
        rows.append({"ticker":t,"close":c,"elig":eligible,"ret":retN})
    if not rows:
        return pd.DataFrame(columns=["ticker","close","elig","ret","rank"])
    tab = pd.DataFrame(rows)
    tab.sort_values(["elig","ret"], ascending=[False, False], inplace=True)
    tab["rank"] = np.arange(1, len(tab)+1)
    return tab

# ============== Gates ==============
def _cross_up(a: pd.Series, b: pd.Series, d: pd.Timestamp) -> bool:
    try:
        return bool((a.shift(1).loc[d] <= b.shift(1).loc[d]) and (a.loc[d] > b.loc[d]))
    except Exception:
        return False

def _cross_down(a: pd.Series, b: pd.Series, d: pd.Timestamp) -> bool:
    try:
        return bool((a.shift(1).loc[d] >= b.shift(1).loc[d]) and (a.loc[d] < b.loc[d]))
    except Exception:
        return False

def bullish_entry_bundle(sym: str, d: pd.Timestamp, daily_ind: Dict[str,pd.Series], return_values=False):
    """Strict entry: MACD cross-up + W%R cross-up from <-80 + hist > 0 and rising."""
    try:
        W = float(daily_ind["WILLR"].loc[d])
        W_prev = float(daily_ind["WILLR"].shift(1).loc[d])
        L = float(daily_ind["MACD_LINE"].loc[d])
        S = float(daily_ind["MACD_SIG"].loc[d])
        H = float(daily_ind["MACD_HIST"].loc[d])
        H_prev = float(daily_ind["MACD_HIST"].shift(1).loc[d])
    except Exception:
        vals = (np.nan,np.nan,np.nan,np.nan)
        return (False, *vals) if return_values else False

    conds = [
        _cross_up(daily_ind["MACD_LINE"], daily_ind["MACD_SIG"], d),
        (W_prev < WILLR_ENTER) and (W >= WILLR_ENTER),
        (H > 0.0),
        (H > H_prev)
    ]
    ok = all(conds)
    return (ok, W, L, S, H) if return_values else ok

def extra_entry_filters(sym: str, d: pd.Timestamp, daily_ind: Dict[str,pd.Series], price_df: pd.DataFrame) -> bool:
    # RSI filter
    if USE_RSI_FILTER:
        try:
            r = float(daily_ind["RSI"].loc[d])
            if np.isnan(r) or (r < RSI_MIN): return False
        except Exception:
            return False
    # Volume breakout
    if USE_VOL_BREAKOUT:
        try:
            v = float(price_df.at[d, "Volume"])
            vma = float(daily_ind["VOL_MA"].loc[d])
            if np.isnan(vma) or v < VOL_MULTIPLIER * vma: return False
        except Exception:
            return False
    return True

def is_bullish_now_for_hold(daily_ind: Dict[str, pd.Series], d: pd.Timestamp) -> bool:
    """Bullish state for rank-exit protection (not a cross event)."""
    try:
        L = float(daily_ind["MACD_LINE"].loc[d])
        S = float(daily_ind["MACD_SIG"].loc[d])
        H = float(daily_ind["MACD_HIST"].loc[d])
        H_prev = float(daily_ind["MACD_HIST"].shift(1).loc[d])
        W = float(daily_ind["WILLR"].loc[d])
    except Exception:
        return False
    conds = []
    if BULLISH_HOLD_REQUIRE_LINE_GT_SIG: conds.append(L > S)
    if BULLISH_HOLD_REQUIRE_HIST_POS:    conds.append(H >= 0.0)
    if BULLISH_HOLD_REQUIRE_HIST_RISE:   conds.append(H > H_prev)
    if BULLISH_HOLD_WILLR_LEVEL is not None: conds.append(W > BULLISH_HOLD_WILLR_LEVEL)
    return all(conds)

def gate_exit_hit(sym: str, d: pd.Timestamp, daily_ind: Dict[str,pd.Series]) -> bool:
    if not USE_WR_MACD_EXITGATE:
        return False
    try:
        if EXIT_REQUIRE_CROSS:
            macd_dn = _cross_down(daily_ind["MACD_LINE"], daily_ind["MACD_SIG"], d)
            willr_dn = bool((daily_ind["WILLR"].shift(1).loc[d] > WILLR_EXIT) and
                            (daily_ind["WILLR"].loc[d] <= WILLR_EXIT))
        else:
            macd_dn = bool(daily_ind["MACD_LINE"].loc[d] < daily_ind["MACD_SIG"].loc[d])
            willr_dn = bool(daily_ind["WILLR"].loc[d] <= WILLR_EXIT)
    except Exception:
        return False
    return (macd_dn or willr_dn) if EXIT_GATE_OR else (macd_dn and willr_dn)

# ============== Engine ==============
@dataclass
class SimplePos:
    ticker: str
    entry_date: pd.Timestamp
    entry_price: float
    shares: int
    reason_entry: str
    signal_date: pd.Timestamp
    sig_willr: float
    sig_macd_line: float
    sig_macd_sig: float
    sig_macd_hist: float
    entry_idx: int

@dataclass
class PendingOrder:
    side: str
    ticker: str
    exec_date: pd.Timestamp
    budget: float = 0.0
    reason: str = ""
    signal_date: Optional[pd.Timestamp] = None
    sig_willr: float = np.nan
    sig_macd_line: float = np.nan
    sig_macd_sig: float = np.nan
    sig_macd_hist: float = np.nan

def simulate_engine(data: Dict[str,pd.DataFrame],
                    caches: Dict[str,Dict[str,pd.Series]],
                    start: str, end: str):
    # Timeline
    all_dates = None
    for df in data.values():
        all_dates = df.index if all_dates is None else all_dates.union(df.index)
    dates = all_dates.sort_values()
    dates = dates[(dates>=pd.to_datetime(start)) & (dates<=pd.to_datetime(end))]
    if len(dates)==0: return pd.DataFrame(), pd.Series(dtype=float)

    # Daily indicators for gates
    daily = {
        t: {"MACD_LINE": caches[t]["MACD_LINE"].reindex(dates),
            "MACD_SIG":  caches[t]["MACD_SIG"].reindex(dates),
            "MACD_HIST": caches[t]["MACD_HIST"].reindex(dates),
            "WILLR":     caches[t]["WILLR"].reindex(dates),
            "RSI":       caches[t]["RSI"].reindex(dates),
            "VOL_MA":    caches[t]["VOL_MA"].reindex(dates)}
        for t in data.keys()
    }

    # Rebalance snapshots (only if ranking enabled)
    if RANKING_ENABLED:
        rebal_dates = compute_rebalance_dates(dates, REBALANCE_FREQ)
        rebal_set = set(rebal_dates)
        current_rank = pd.DataFrame()
    else:
        rebal_dates = []
        rebal_set   = set()
        current_rank= pd.DataFrame()

    # State
    cash = STARTING_CAPITAL
    positions: Dict[str, SimplePos] = {}
    pending: List[PendingOrder] = []
    equity_series = []
    trades_rows = []

    def equity_on_close(d):
        val = cash
        for sym, pos in positions.items():
            if d in data[sym].index:
                val += float(data[sym].at[d,"Close"]) * pos.shares
        return float(val)

    for d in dates:
        # 1) Open: execute queued (sells → buys)
        if pending:
            todays = [o for o in pending if o.exec_date == d]
            if todays:
                # Sells first
                for o in [x for x in todays if x.side=="sell"]:
                    sym = o.ticker
                    if sym not in positions or d not in data[sym].index: continue
                    open_px = float(data[sym].at[d,"Open"])
                    eff_sell = open_px * (1 - SLIPPAGE_BPS/10_000.0)
                    pos = positions[sym]
                    t_buy  = pos.entry_price * pos.shares
                    t_sell = eff_sell * pos.shares
                    fees = calc_fees(t_buy, t_sell)
                    pnl  = (eff_sell - pos.entry_price) * pos.shares - fees
                    cash += (eff_sell * pos.shares) - fees
                    trades_rows.append({
                        "ticker": sym,
                        "entry_date": pos.entry_date.date(),
                        "exit_date": d.date(),
                        "entry_price": round(pos.entry_price,2),
                        "exit_price":  round(eff_sell,2),
                        "shares": pos.shares,
                        "gross_pnl": round((eff_sell - pos.entry_price) * pos.shares,2),
                        "fees": round(fees,2),
                        "net_pnl": round(pnl,2),
                        "reason_entry": pos.reason_entry,
                        "reason_exit": o.reason or "exit_gate",
                        "signal_date": pos.signal_date.date(),
                        "sig_willr": pos.sig_willr,
                        "sig_macd_line": pos.sig_macd_line,
                        "sig_macd_sig": pos.sig_macd_sig,
                        "sig_macd_hist": pos.sig_macd_hist,
                    })
                    positions.pop(sym, None)
                # Buys then
                for o in [x for x in todays if x.side=="buy"]:
                    sym = o.ticker
                    if sym in positions or d not in data[sym].index: continue
                    open_px = float(data[sym].at[d,"Open"])
                    eff_buy = open_px * (1 + SLIPPAGE_BPS/10_000.0)
                    per_cap = max(0.0, o.budget)
                    shares = math.floor(per_cap / eff_buy)
                    if shares <= 0: continue
                    cost = eff_buy * shares
                    fees = calc_fees(cost, 0.0)
                    if cost + fees > cash:
                        shares = math.floor((cash / (1 + SLIPPAGE_BPS/10_000.0)) / eff_buy)
                        if shares <= 0: continue
                        cost = eff_buy * shares
                        fees = calc_fees(cost, 0.0)
                        if cost + fees > cash: continue
                    cash -= (cost + fees)
                    positions[sym] = SimplePos(
                        ticker=sym, entry_date=d, entry_price=eff_buy, shares=shares,
                        reason_entry=o.reason or "entry_bull_bundle",
                        signal_date=o.signal_date or d, sig_willr=o.sig_willr,
                        sig_macd_line=o.sig_macd_line, sig_macd_sig=o.sig_macd_sig, sig_macd_hist=o.sig_macd_hist,
                        entry_idx=dates.get_loc(d)
                    )
                pending = [o for o in pending if o.exec_date != d]

        # 2) Intraday stop
        if USE_STOP_LOSS and positions:
            to_stop=[]
            for sym, pos in positions.items():
                if d not in data[sym].index: continue
                day_low = float(data[sym].at[d,"Low"])
                stop_px = pos.entry_price * (1.0 - STOP_LOSS_PCT)
                if not np.isnan(day_low) and day_low <= stop_px:
                    eff_sell = stop_px * (1 - SLIPPAGE_BPS/10_000.0)
                    t_buy  = pos.entry_price * pos.shares
                    t_sell = eff_sell * pos.shares
                    fees = calc_fees(t_buy, t_sell)
                    pnl  = (eff_sell - pos.entry_price) * pos.shares - fees
                    cash += (eff_sell * pos.shares) - fees
                    trades_rows.append({
                        "ticker": sym,
                        "entry_date": pos.entry_date.date(),
                        "exit_date": d.date(),
                        "entry_price": round(pos.entry_price,2),
                        "exit_price":  round(eff_sell,2),
                        "shares": pos.shares,
                        "gross_pnl": round((eff_sell - pos.entry_price) * pos.shares,2),
                        "fees": round(fees,2),
                        "net_pnl": round(pnl,2),
                        "reason_entry": pos.reason_entry,
                        "reason_exit": f"stop_loss_{int(STOP_LOSS_PCT*100)}pct",
                        "signal_date": pos.signal_date.date(),
                        "sig_willr": pos.sig_willr,
                        "sig_macd_line": pos.sig_macd_line,
                        "sig_macd_sig": pos.sig_macd_sig,
                        "sig_macd_hist": pos.sig_macd_hist,
                    })
                    to_stop.append(sym)
            for sym in to_stop: positions.pop(sym, None)

        # 3) Rebalance (only if ranking enabled)
        if RANKING_ENABLED and d in rebal_set:
            current_rank = build_rank_table(d, data, caches)
            log(f"[rebalance] rank snapshot built for {d.date()} (rows={len(current_rank)})")
            if not current_rank.empty and RANK_EXIT_THRESHOLD is not None:
                rank_map = {row.ticker:int(row.rank) for row in current_rank.itertuples()}
                elig_map = {row.ticker:bool(row.elig) for row in current_rank.itertuples()}
                idx_today = dates.get_loc(d)
                exec_d = dates[idx_today+1] if (idx_today+1) < len(dates) else None
                if exec_d is not None:
                    for sym in list(positions.keys()):
                        r = rank_map.get(sym, 10**9)
                        elig = elig_map.get(sym, False)
                        # protect bullish positions if enabled
                        if PRIORITIZE_BULLISH_ON_REBAL and is_bullish_now_for_hold(daily[sym], d):
                            log(f"[rebalance] protect {sym}: bullish_now; skip rank exit (r={r}, eligible={elig})")
                            continue
                        if (r > RANK_EXIT_THRESHOLD) or (not elig):
                            if not any(o.side=="sell" and o.ticker==sym and o.exec_date==exec_d for o in pending):
                                pending.append(PendingOrder(side="sell", ticker=sym, exec_date=exec_d,
                                                            reason=f"rank_exit(r={r}, eligible={elig})"))

        # 4) Daily technical exit
        if USE_WR_MACD_EXITGATE and positions:
            idx_today = dates.get_loc(d)
            exec_d = dates[idx_today+1] if (idx_today+1) < len(dates) else None
            if exec_d is not None:
                for sym, pos in positions.items():
                    if gate_exit_hit(sym, d, daily[sym]):
                        if not any(o.side=="sell" and o.ticker==sym and o.exec_date==exec_d for o in pending):
                            pending.append(PendingOrder(side="sell", ticker=sym, exec_date=exec_d,
                                                        reason="exit_gate(W%R+MACD)"))

        # 5) Daily entries
        idx_today = dates.get_loc(d)
        exec_d = dates[idx_today+1] if (idx_today+1) < len(dates) else None
        if exec_d is not None:
            have = set(positions.keys())
            future_hold = len(positions) - sum(1 for o in pending if o.exec_date==exec_d and o.side=="sell")
            need = max(0, RANK_TOP_N - future_hold)
            if need > 0:
                # build candidate list depending on ranking mode
                cands: List[str] = []
                if RANKING_ENABLED:
                    if 'current_rank' in locals() and not current_rank.empty:
                        # eligible & within Top-N scanning window
                        scan = current_rank[current_rank["elig"]].copy()
                        scan.sort_values("rank", inplace=True)
                        # we scan up to max(rank) to find signals; cap is RANK_TOP_N * 3 to allow replacements
                        limit = max(RANK_TOP_N * 3, 50)
                        scan = scan[scan["rank"] <= limit]
                        cands = [row.ticker for row in scan.itertuples()]
                else:
                    # first-come-first-serve: UNIVERSE order
                    cands = list(UNIVERSE)

                if cands:
                    per_cap = equity_on_close(d) / max(1, RANK_TOP_N)
                    for sym in cands:
                        if need <= 0: break
                        if sym in have: continue
                        if any(o.side=="buy" and o.ticker==sym and o.exec_date==exec_d for o in pending): continue
                        if d not in data[sym].index: continue

                        # If ranking disabled, still enforce base eligibility (within 52w & > SMA200)
                        try:
                            c = float(data[sym].at[d, "Close"])
                            h = float(caches[sym]["HIGH_252"].loc[d])
                            s = float(caches[sym]["SMA_200"].loc[d])
                            elig = (c >= (1.0 - WITHIN_52W)*h) and (c > s)
                        except Exception:
                            elig = False
                        if not elig: continue

                        ok, W, L, S, H = bullish_entry_bundle(sym, d, daily[sym], return_values=True)
                        if not ok: continue
                        if not extra_entry_filters(sym, d, daily[sym], data[sym]): continue

                        pending.append(PendingOrder(
                            side="buy", ticker=sym, exec_date=exec_d, budget=per_cap,
                            reason=(f"entry_bull_bundle({'ranked' if RANKING_ENABLED else 'fcfs'};"
                                    f" elig(within{int(WITHIN_52W*100)}%+SMA200)"
                                    f"{' +RSI' if USE_RSI_FILTER else ''}{' +VOL' if USE_VOL_BREAKOUT else ''})"),
                            signal_date=d, sig_willr=W, sig_macd_line=L, sig_macd_sig=S, sig_macd_hist=H
                        ))
                        need -= 1

        # EOD equity mark
        equity_series.append((d, equity_on_close(d)))

    # Liquidate at final close
    if len(dates) > 0:
        last_d = dates[-1]
        for sym, pos in list(positions.items()):
            if last_d in data[sym].index:
                px = float(data[sym].at[last_d, "Close"])
            else:
                continue
            eff_sell = px * (1 - SLIPPAGE_BPS/10_000.0)
            t_buy  = pos.entry_price * pos.shares
            t_sell = eff_sell * pos.shares
            fees = calc_fees(t_buy, t_sell)
            pnl  = (eff_sell - pos.entry_price) * pos.shares - fees
            trades_rows.append({
                "ticker": sym,
                "entry_date": pos.entry_date.date(),
                "exit_date": last_d.date(),
                "entry_price": round(pos.entry_price,2),
                "exit_price":  round(eff_sell,2),
                "shares": pos.shares,
                "gross_pnl": round((eff_sell - pos.entry_price) * pos.shares,2),
                "fees": round(fees,2),
                "net_pnl": round(pnl,2),
                "reason_entry": pos.reason_entry,
                "reason_exit": "liq_eod",
                "signal_date": pos.signal_date.date(),
                "sig_willr": pos.sig_willr,
                "sig_macd_line": pos.sig_macd_line,
                "sig_macd_sig": pos.sig_macd_sig,
                "sig_macd_hist": pos.sig_macd_hist,
            })
        positions.clear()
        equity_series.append((last_d, equity_on_close(last_d)))

    eq = pd.Series({d: v for d, v in equity_series}).sort_index()
    trades_df = pd.DataFrame(trades_rows)
    return trades_df, eq

# ============== Metrics & Writers ==============
def compute_metrics(trades_df: pd.DataFrame, equity: pd.Series, start: str, end: str):
    if equity is None or len(equity) == 0:
        return {"trades":0,"win_rate_pct":0.0,"profit_factor":0.0,"sharpe":0.0,"cagr_pct":0.0,"max_dd_pct":0.0}
    eq = equity.sort_index().ffill()
    ret = eq.pct_change().dropna()
    sharpe = (ret.mean()/ret.std()) * np.sqrt(252) if ret.std() > 0 else 0.0
    rollmax = eq.cummax()
    max_dd_pct = float(((eq/rollmax - 1.0).min()) * 100.0)
    years = max(1e-9, (pd.to_datetime(end) - pd.to_datetime(start)).days/365.25)
    cagr = (eq.iloc[-1]/eq.iloc[0])**(1/years) - 1 if eq.iloc[0] > 0 else 0.0
    if trades_df is None or trades_df.empty:
        trades=0; wr=0.0; pf=0.0
    else:
        trades = len(trades_df)
        wins = (trades_df["net_pnl"] > 0).sum()
        wr = 100.0 * wins / max(1, trades)
        gains = trades_df["net_pnl"].clip(lower=0).sum()
        losses= -trades_df["net_pnl"].clip(upper=0).sum()
        pf = (gains / losses) if losses > 0 else (np.inf if gains > 0 else 0.0)
    return {"trades":int(trades),"win_rate_pct":float(wr),"profit_factor":float(pf),
            "sharpe":float(sharpe),"cagr_pct":float(cagr*100.0),"max_dd_pct":float(max_dd_pct)}

def write_trades_csv(df_trades: pd.DataFrame, filepath: str):
    cols = ["ticker","entry_date","exit_date","hold_days","entry_price","exit_price",
            "shares","gross_pnl","fees","net_pnl",
            "reason_entry","reason_exit",
            "start_capital","ranking_enabled","rebalance_freq","rank_top_n","rank_exit_threshold",
            "rank_lookback_days","within_52w","prioritize_bullish_on_rebal",
            "use_rsi_filter","rsi_len","rsi_min",
            "use_vol_breakout","vol_lookback","vol_multiplier",
            "use_wr_macd_exitgate","exit_require_cross","exit_gate_or",
            "use_stop_loss","stop_loss_pct",
            "signal_date","sig_willr","sig_macd_line","sig_macd_sig","sig_macd_hist"]
    if df_trades is None or df_trades.empty:
        pd.DataFrame(columns=cols).to_csv(filepath, index=False); log(f"Wrote (empty): {filepath}"); return
    out = df_trades.copy()
    out["entry_date"]=pd.to_datetime(out["entry_date"]); out["exit_date"]=pd.to_datetime(out["exit_date"])
    out["hold_days"]=(out["exit_date"]-out["entry_date"]).dt.days
    out["start_capital"]=STARTING_CAPITAL
    out["ranking_enabled"]=RANKING_ENABLED
    out["rebalance_freq"]=REBALANCE_FREQ
    out["rank_top_n"]=RANK_TOP_N
    out["rank_exit_threshold"]=RANK_EXIT_THRESHOLD
    out["rank_lookback_days"]=RANK_LOOKBACK_DAYS
    out["within_52w"]=WITHIN_52W
    out["prioritize_bullish_on_rebal"]=PRIORITIZE_BULLISH_ON_REBAL
    out["use_rsi_filter"]=USE_RSI_FILTER
    out["rsi_len"]=RSI_LEN
    out["rsi_min"]=RSI_MIN
    out["use_vol_breakout"]=USE_VOL_BREAKOUT
    out["vol_lookback"]=VOL_LOOKBACK
    out["vol_multiplier"]=VOL_MULTIPLIER
    out["use_wr_macd_exitgate"]=USE_WR_MACD_EXITGATE
    out["exit_require_cross"]=EXIT_REQUIRE_CROSS
    out["exit_gate_or"]=EXIT_GATE_OR
    out["use_stop_loss"]=USE_STOP_LOSS
    out["stop_loss_pct"]=STOP_LOSS_PCT
    out = out.reindex(columns=cols)
    out.to_csv(filepath, index=False); log(f"Wrote: {filepath} (rows={len(out)})")

def write_monthly_pnl_csv(df_trades: pd.DataFrame, filepath: str):
    cols = ["month","net_pnl"]
    if df_trades is None or df_trades.empty:
        pd.DataFrame(columns=cols).to_csv(filepath, index=False); log(f"Wrote (empty): {filepath}"); return
    df = df_trades.copy()
    df["exit_date"]=pd.to_datetime(df["exit_date"])
    monthly = df.groupby(df["exit_date"].dt.to_period("M"))["net_pnl"].sum().reset_index()
    monthly["month"]=monthly["exit_date"].astype(str)
    monthly = monthly[["month","net_pnl"]]
    monthly.to_csv(filepath, index=False); log(f"Wrote: {filepath} (rows={len(monthly)})")

# ============== Main ==============
def run_pipeline(force_refresh: bool=False):
    log("=== Loading data (cache) ===")
    data = load_or_download_prices(UNIVERSE, TRAIN_START, TEST_END, force_refresh=force_refresh)
    if not data: raise RuntimeError("No stock data available.")
    log(f"Loaded {len(data)} symbols.")
    log("=== Building indicators ===")
    caches = build_indicator_caches(data, RANK_LOOKBACK_DAYS, force_refresh=force_refresh)

    def run_period(label, start, end):
        log(f"=== Simulating {label} ({start}..{end}) ===")
        trades_df, eq = simulate_engine(data, caches, start, end)
        log(f"{label} trades: {0 if trades_df is None else len(trades_df)}")
        m = compute_metrics(trades_df, eq, start, end)
        log(f"Final {label} metrics: {fmt_metrics(m)}")
        return trades_df, eq, m

    log("=== Parameters ===")
    log(f"  Ranking: enabled={RANKING_ENABLED}, rebalance={REBALANCE_FREQ}, TopN(cap)={RANK_TOP_N}, exit>rank={RANK_EXIT_THRESHOLD}")
    log(f"  Eligibility: within {int(WITHIN_52W*100)}% of 52w high & > SMA200 (applied in both modes)")
    log(f"  Entry: MACD↑ + W%R↑(from {WILLR_ENTER}) + hist>0 & rising"
        f"{' +RSI' if USE_RSI_FILTER else ''}{' +VOL' if USE_VOL_BREAKOUT else ''}")
    log(f"  Exit: WR/MACD gate (OR={EXIT_GATE_OR}, cross={EXIT_REQUIRE_CROSS}), StopLoss={STOP_LOSS_PCT*100:.1f}%")
    log(f"  Fees={APPLY_FEES}, Slippage={SLIPPAGE_BPS}bps, ProtectBullishOnRebal={PRIORITIZE_BULLISH_ON_REBAL and RANKING_ENABLED}")

    train_trades, train_eq, _ = run_period("TRAIN", TRAIN_START, TRAIN_END)
    test_trades,  test_eq,  _ = run_period("TEST",  TEST_START,  TEST_END)

    log("=== Writing CSVs ===")
    write_trades_csv(train_trades, os.path.join(OUTDIR, "trades_train.csv"))
    write_trades_csv(test_trades,  os.path.join(OUTDIR, "trades_test.csv"))
    write_monthly_pnl_csv(train_trades, os.path.join(OUTDIR, "pnl_monthly_train.csv"))
    write_monthly_pnl_csv(test_trades,  os.path.join(OUTDIR, "pnl_monthly_test.csv"))
    log("=== Done ===")

if __name__ == "__main__":
    run_pipeline(force_refresh=FORCE_REFRESH)


[10:11:36] === Loading data (cache) ===
[10:11:44] [cache] downloading 25/500: AGARWALEYE.NS
[10:11:53] [cache] downloading 50/500: ASTRAL.NS
[10:12:03] [cache] downloading 75/500: BEML.NS
[10:12:12] [cache] downloading 100/500: CCL.NS
[10:12:22] [cache] downloading 125/500: CRAFTSMAN.NS
[10:12:30] [cache] downloading 150/500: EIHOTEL.NS
[10:12:38] [cache] downloading 175/500: GILLETTE.NS
[10:12:47] [cache] downloading 200/500: HCLTECH.NS
[10:12:57] [cache] downloading 225/500: IEX.NS
[10:13:05] [cache] downloading 250/500: IRFC.NS
[10:13:14] [cache] downloading 275/500: KALYANKJIL.NS
[10:13:23] [cache] downloading 300/500: LTFOODS.NS
[10:13:31] [cache] downloading 325/500: MOTILALOFS.NS
[10:13:39] [cache] downloading 350/500: NTPC.NS
[10:13:47] [cache] downloading 375/500: PNB.NS
[10:13:57] [cache] downloading 400/500: RITES.NS
[10:14:04] [cache] downloading 425/500: SKFINDIA.NS
[10:14:13] [cache] downloading 450/500: TATAPOWER.NS
[10:14:22] [cache] downloading 475/500: UNIONBANK.NS
[